In [34]:
import os
import pandas as pd

In [35]:
list_destination = {
    "1": "Pantai Joko Mursodo",
    "2": "Pantai Pengkolan Kandang Semangkon",
    "3": "Pantai Kutang",
    "4": "Taman Wisata dan Perkemahan Bumi Moronyamplung",
    "5": "Gunung Mas Mantup",
    "6": "Goa Maharani",
    "7": "Wisata Gunung Pegat",
    "8": "Wisata Air Hangat Mbrumbung",
    "9": "Taman Ekspresi Kendalifornia",
    "10": "Waduk Gondang",
    "11": "Kolam Renang Keraton",
    "12": "Wisata Edukasi Gondang Outbond",
    "13": "INDONESIA ISLAMIC ART MUSEUM",
    "14": "Wisata Bahari Lamongan",
    "15": "G-Park",
    "16": "Makam Sendang Duwur",
    "17": "Makam Sunan Drajat",
    "18": "Masjid Namira",
    "19": "Masjid Ki Bagus Hadikusumo",
    "20": "Makam Syekh Maulana Ishaq"
}

# Folder tempat file Excel disimpan
raw_folder_path = "../data/raw"
processed_folder_path = "../data/processed"

def convert_xlsx_to_csv(file_name):
    try:
        os.makedirs(processed_folder_path, exist_ok=True)

        base_name = os.path.splitext(os.path.basename(file_name))[0]
        input_path = os.path.join(raw_folder_path, base_name + ".xlsx")
        output_path = os.path.join(processed_folder_path, base_name + ".csv")

        df_convert = pd.read_excel(input_path)
        df_convert.to_csv(output_path, index=False)

        print(f"[✓] Berhasil mengonversi {input_path} ke {output_path}")
    except Exception as e_convert:
        print(f"[!] Gagal mengonversi {file_name} - Error: {e_convert}")

missing_files = []

for key, name in list_destination.items():
    xlsx_filename = f"{name}.xlsx"
    xlsx_path = os.path.join(raw_folder_path, xlsx_filename)

    if not os.path.isfile(xlsx_path):
        missing_files.append(xlsx_filename)
    else:
        try:
            df = pd.read_excel(xlsx_path)
            print(f"[✓] File ditemukan dan bisa dibaca: {xlsx_filename}")
            convert_xlsx_to_csv(xlsx_filename)
        except Exception as e:
            print(f"[!] File ditemukan tapi gagal dibaca: {xlsx_filename} - Error: {e}")

if missing_files:
    print("\n[!] File berikut tidak ditemukan:")
    for file in missing_files:
        print(f"- {file}")
else:
    print("\n✅ Semua file ditemukan dan bisa dibaca.")


[✓] File ditemukan dan bisa dibaca: Pantai Joko Mursodo.xlsx
[✓] Berhasil mengonversi ../data/raw\Pantai Joko Mursodo.xlsx ke ../data/processed\Pantai Joko Mursodo.csv
[✓] File ditemukan dan bisa dibaca: Pantai Pengkolan Kandang Semangkon.xlsx
[✓] Berhasil mengonversi ../data/raw\Pantai Pengkolan Kandang Semangkon.xlsx ke ../data/processed\Pantai Pengkolan Kandang Semangkon.csv
[✓] File ditemukan dan bisa dibaca: Pantai Kutang.xlsx
[✓] Berhasil mengonversi ../data/raw\Pantai Kutang.xlsx ke ../data/processed\Pantai Kutang.csv
[✓] File ditemukan dan bisa dibaca: Taman Wisata dan Perkemahan Bumi Moronyamplung.xlsx
[✓] Berhasil mengonversi ../data/raw\Taman Wisata dan Perkemahan Bumi Moronyamplung.xlsx ke ../data/processed\Taman Wisata dan Perkemahan Bumi Moronyamplung.csv
[✓] File ditemukan dan bisa dibaca: Gunung Mas Mantup.xlsx
[✓] Berhasil mengonversi ../data/raw\Gunung Mas Mantup.xlsx ke ../data/processed\Gunung Mas Mantup.csv
[✓] File ditemukan dan bisa dibaca: Goa Maharani.xlsx
[✓] 

In [36]:
# Daftar nama kolom tetap (17 kolom)
column_names = [
    "nomor",
    "escape",
    "relaxation",
    "play",
    "strengthening_family_bonds",
    "prestige",
    "social_interaction",
    "romance",
    "educational_opportunity",
    "self_fulfillment",
    "wish_fulfillment",
    "infrastruktur_pariwisata",
    "edukasi",
    "alam_sekitar",
    "atraksi_budaya_dan_sejarah",
    "kuliner",
    "aktivitas_wisata",
]

aktivitas_wisata = set()

def get_all_activities():
    global aktivitas_wisata

    for key, name in list_destination.items():
        filename = f"{name}.csv"
        file_path = os.path.join(processed_folder_path, filename)

        if not os.path.exists(file_path):
            print(f"[!] File tidak ditemukan: {file_path}")
            continue

        try:
            df = pd.read_csv(file_path)
            if "aktivitas_wisata" in df.columns:
                for row in df["aktivitas_wisata"].dropna():
                    items = [item.strip().lower().replace(" ", "_") for item in str(row).split(",")]
                    aktivitas_wisata.update(items)
        except Exception as e:
            print(f"[!] Gagal membaca {filename} - Error: {e}")

    aktivitas_list = sorted(aktivitas_wisata)
    return aktivitas_list

def clean_and_get_index(csv_path):
    try:
        df_csv = pd.read_csv(csv_path)

        # Langsung hapus 5 kolom pertama (karena header sudah benar)
        df_csv.drop(df_csv.columns[:5], axis=1, inplace=True)

        # Reset index dan tambahkan kolom 'nomor' di depan
        df_csv.reset_index(drop=True, inplace=True)
        df_csv.insert(0, "nomor", range(1, len(df_csv) + 1))

        # Tetapkan nama kolom sesuai daftar tetap
        df_csv.columns = column_names[:len(df_csv.columns)]

        return df_csv
    except Exception as e:
        print(f"[!] Error saat membersihkan {csv_path}: {e}")
        return None


def setting_push_factor(df_push_factor):
    # Misalnya kolom push factor ada di index 1 sampai 10
    for i in range(1, 11):
        df_push_factor.iloc[:, i] = df_push_factor.iloc[:, i].apply(lambda x: 1 if str(x).strip().lower() == "bisa" else 0)

    return df_push_factor

def setting_pull_factor(df_pull_factor):
    # Misalnya kolom pull factor ada di index 11 sampai 17
    for i in range(11, 16):
        df_pull_factor.iloc[:, i] = df_pull_factor.iloc[:, i].apply(lambda x: 1 if str(x).strip().lower() == "setuju" else 0)

    return df_pull_factor

In [37]:
for key, name in list_destination.items():
    filename = f"{name}.csv"
    file_path = os.path.join(processed_folder_path, filename)

    if not os.path.exists(file_path):
        print(f"[!] File tidak ditemukan: {file_path}")
        continue

    df = clean_and_get_index(file_path)

    if df is not None:
        # Hapus kolom yang tidak ada dalam daftar nama kolom tetap
        df = df.loc[:, df.columns.isin(column_names)]

        # Mengatur nilai push factor
        df = setting_push_factor(df)
        # Mengatur nilai pull factor
        df = setting_pull_factor(df)

    if df is not None and not df.empty:
        # Timpa file lama dengan versi yang sudah dibersihkan
        df.to_csv(file_path, index=False)
        print(f"[✓] File {filename} berhasil dibersihkan dan ditimpa.")
    else:
        print(f"[!] File {filename} kosong atau gagal dibersihkan.")


get_all_activities()

[✓] File Pantai Joko Mursodo.csv berhasil dibersihkan dan ditimpa.
[✓] File Pantai Pengkolan Kandang Semangkon.csv berhasil dibersihkan dan ditimpa.
[✓] File Pantai Kutang.csv berhasil dibersihkan dan ditimpa.
[✓] File Taman Wisata dan Perkemahan Bumi Moronyamplung.csv berhasil dibersihkan dan ditimpa.
[✓] File Gunung Mas Mantup.csv berhasil dibersihkan dan ditimpa.
[✓] File Goa Maharani.csv berhasil dibersihkan dan ditimpa.
[✓] File Wisata Gunung Pegat.csv berhasil dibersihkan dan ditimpa.
[✓] File Wisata Air Hangat Mbrumbung.csv berhasil dibersihkan dan ditimpa.
[✓] File Taman Ekspresi Kendalifornia.csv berhasil dibersihkan dan ditimpa.
[✓] File Waduk Gondang.csv berhasil dibersihkan dan ditimpa.
[✓] File Kolam Renang Keraton.csv berhasil dibersihkan dan ditimpa.
[✓] File Wisata Edukasi Gondang Outbond.csv berhasil dibersihkan dan ditimpa.
[✓] File INDONESIA ISLAMIC ART MUSEUM.csv berhasil dibersihkan dan ditimpa.
[✓] File Wisata Bahari Lamongan.csv berhasil dibersihkan dan ditimpa.


['belajar',
 'berbelanja',
 'berenang',
 'berendam',
 'berinteraksi_dengan_satwa',
 'berkemah',
 'bermain_wahana',
 'berziarah',
 'makan',
 'melihat_pemandangan',
 'memancing',
 'mengambil_foto']

In [38]:
# Pastikan aktivitas_wisata sudah dalam bentuk snake_case
aktivitas_wisata = [a.replace(" ", "_").lower() for a in aktivitas_wisata]

for key, name in list_destination.items():
    filename = f"{name}.csv"
    file_path = os.path.join(processed_folder_path, filename)

    if not os.path.exists(file_path):
        print(f"[!] File tidak ditemukan: {file_path}")
        continue

    df = pd.read_csv(file_path)

    if "aktivitas_wisata" not in df.columns:
        print(f"[!] Kolom 'aktivitas_wisata' tidak ditemukan di {filename}")
        continue

    # Normalisasi: ubah ke lowercase
    df["aktivitas_wisata"] = df["aktivitas_wisata"].fillna("").str.lower()

    # Tambahkan kolom baru untuk setiap aktivitas
    for activity in aktivitas_wisata:
        df[activity] = df["aktivitas_wisata"].apply(
            lambda x: 1 if activity in [a.strip().replace(" ", "_") for a in x.split(",")] else 0
        )

    # Simpan ulang file
    df.to_csv(file_path, index=False)
    print(f"[✓] File diperbarui: {file_path}")



[✓] File diperbarui: ../data/processed\Pantai Joko Mursodo.csv
[✓] File diperbarui: ../data/processed\Pantai Pengkolan Kandang Semangkon.csv
[✓] File diperbarui: ../data/processed\Pantai Kutang.csv
[✓] File diperbarui: ../data/processed\Taman Wisata dan Perkemahan Bumi Moronyamplung.csv
[✓] File diperbarui: ../data/processed\Gunung Mas Mantup.csv
[✓] File diperbarui: ../data/processed\Goa Maharani.csv
[✓] File diperbarui: ../data/processed\Wisata Gunung Pegat.csv
[✓] File diperbarui: ../data/processed\Wisata Air Hangat Mbrumbung.csv
[✓] File diperbarui: ../data/processed\Taman Ekspresi Kendalifornia.csv
[✓] File diperbarui: ../data/processed\Waduk Gondang.csv
[✓] File diperbarui: ../data/processed\Kolam Renang Keraton.csv
[✓] File diperbarui: ../data/processed\Wisata Edukasi Gondang Outbond.csv
[✓] File diperbarui: ../data/processed\INDONESIA ISLAMIC ART MUSEUM.csv
[✓] File diperbarui: ../data/processed\Wisata Bahari Lamongan.csv
[✓] File diperbarui: ../data/processed\G-Park.csv
[✓] Fi

In [39]:
# dapatkan jumlah data paling sedikit
for key, name in list_destination.items():
    filename = f"{name}.csv"
    file_path = os.path.join(processed_folder_path, filename)

    if not os.path.exists(file_path):
        print(f"[!] File tidak ditemukan: {file_path}")
        continue

    df = pd.read_csv(file_path)

In [40]:
all_data = []

# Ambil data sebanyak min_rows dari setiap file
for key, name in list_destination.items():
    filename = f"{name}.csv"
    file_path = os.path.join(processed_folder_path, filename)

    if not os.path.exists(file_path):
        continue

    df = pd.read_csv(file_path)

    # Tambahkan kolom Nama Wisata
    df["Nama Wisata"] = name

    # Pindahkan kolom Nama Wisata ke depan
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index("Nama Wisata")))
    df = df[cols]

    # Ambil hanya sebanyak min_rows
    df = df.head(30)

    all_data.append(df)

# Gabungkan semua DataFrame menjadi satu
combined_df = pd.concat(all_data, ignore_index=True)

# Hapus kolom nomor jika ada
if "nomor" in combined_df.columns:
    combined_df = combined_df.drop(columns=["nomor"])

# Simpan ke file CSV
combined_df.to_csv(os.path.join("../data", "dataset.csv"), index=False)
print("[✓] Dataset berhasil dibuat dan disimpan.")

[✓] Dataset berhasil dibuat dan disimpan.
